In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/sleepy-driver-eeg-brainwave-data/acquiredDataset.csv')
df.info()

In [ ]:
#check null values
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.heatmap(df.corr())

**Trying to find correlated features with higher values then, Manupulatiogn them.**

In [ ]:
df.corr()

In [ ]:
df.describe()

In [ ]:
c = df.corr().abs()

In [ ]:
s = c.unstack()

In [ ]:
so = s.sort_values(kind="quicksort")

In [ ]:
so.argmax()

In [ ]:
high_corr = []

In [ ]:
for index, value in so.items():
    if value > 0.5:
        if index[0] == index[1]:
            pass
        else:
            print(f"{index[0]} , {index[1]} = {value}")
            high_corr.append([index[0],index[1],value])

In [ ]:
high_corr

**Defining a self-made factor for them**

In [ ]:
scored_corr = []
for item in high_corr:
    score = item[2] / 0.7
    item.append(score)
    scored_corr.append(item)

In [ ]:
scored_corr

In [ ]:
cols = df.columns
cols

In [ ]:
y = df['classification']

In [ ]:
df.head(5)

In [ ]:
x = df.drop(columns = ['classification'])

In [ ]:
x.head(5)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
len(cols)

In [ ]:
x.head(5)

Making Engineered Features

In [ ]:
x.shape

In [ ]:
ncols = cols[:10]

In [ ]:
ncols

Multiply interest rate

In [ ]:
m = x.shape[1]
for i in range(0,m):
    for j in range(1,m):
        if x.columns[i] == scored_corr[j][0] and x.columns[j] == scored_corr[j][1]:
            x[f'{x.columns[i]}_x_{x.columns[j]}'] =  x[x.columns[i]] * x[x.columns[j]] * scored_corr[j][3]
        
        else:
            x[f'{x.columns[i]}_x_{x.columns[j]}'] =  x[x.columns[i]] * x[x.columns[j]]

In [ ]:
x

In [ ]:
columns = x.columns
columns

In [ ]:
scaler = MinMaxScaler()
# scaled = scaler.fit_transform(x, axis=1)

In [ ]:
scaled.shape

In [ ]:
scaled = pd.DataFrame(x, columns=[columns])

In [ ]:
scaled.head(5)

In [ ]:
x.shape

In [ ]:
for item in x.columns:
    print(item)
    x[f'{item}'] = x[f'{item}']/np.max(x[f'{item}'])

In [ ]:
x.head()

## Split Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense 
from tensorflow.keras import Sequential, Input

In [ ]:
model = Sequential([
    Input(shape=(100,)),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-7), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=500)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Accuracy')

In [ ]:
plt.plot(history.history['loss'])
plt.title('Loss')